### Initialization

Let's first set up the python environment and define the configuration file

In [1]:
# Import required standard modules
import shutil
import sys
from pathlib import Path

import numpy as np

# Import required icepy4d4D modules
from icepy4d import classes as icepy4d_classes
from icepy4d.classes.epoch import Epoch, Epoches
from icepy4d import matching
from icepy4d import sfm
from icepy4d import io
from icepy4d import utils as icepy4d_utils
from icepy4d.metashape import metashape as MS
from icepy4d.utils import initialization as inizialization

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Define the path to the configuration file
CFG_FILE = "config/config_2022.yaml"

Inizialize all the required variables

In [3]:
# Parse the configuration file
cfg_file = Path(CFG_FILE)
cfg = inizialization.parse_cfg(cfg_file)

# Initialize the timer and logger
timer_global = icepy4d_utils.AverageTimer()
logger = icepy4d_utils.get_logger()

# Get the list of cameras from the configuration file
cams = cfg.cams

# Get the list of images from the configuration file
images, epoch_dict = inizialization.initialize_image_ds(cfg)

# Initialize an empty Epoches object to store the results of each epoch
epoches = Epoches(starting_epoch=cfg.proc.epoch_to_process[0])


ICEpy4D
Image-based Continuos monitoring of glaciers' Evolution with low-cost stereo-cameras and Deep Learning photogrammetry
2023 - Francesco Ioli - francesco.ioli@polimi.it

2023-08-29 18:38:13 | [INFO    ] Configuration file: config_2022
2023-08-29 18:38:13 | [INFO    ] Epoch_to_process set to a pair of values. Expanding it for a range of epoches from epoch 0 to 158.
2023-08-29 18:38:13 | [INFO    ] Image datastores created successfully.


### Stereo Processing

The stereo processing is carried out for each epoch in order to find matched features, estimating camera pose, and triangulating the 3D points. 
The output of this step is a set of 3D points and their corresponding descriptors.

The processing for all the epoches is then iterated in a big loop.

#### Load or create a new Epoch object

In [4]:
# Initialize a timer to measure the processing time
timer = icepy4d_utils.AverageTimer()

# Get epoch id to process
ep = cfg.proc.epoch_to_process[0]

# Define paths to the epoch directory
epoch_name = epoch_dict[ep]
epochdir = cfg.paths.results_dir / epoch_name

# Load an existing epoch or create a new one
if cfg.proc.load_existing_results:
    try:
        # Load existing epcoh from pickle file
        epoch = Epoch.read_pickle(epochdir / f"{epoch_name}.pickle")

    except:
        logger.error(
            f"Unable to load epoch {epoch_name} from pickle file. Creating new epoch..."
        )
        epoch = inizialization.initialize_epoch(
            cfg=cfg, images=images, epoch_id=ep, epoch_dir=epochdir
        )

else:
    # Create new epoch object
    epoch = inizialization.initialize_epoch(
        cfg=cfg, images=images, epoch_id=ep, epoch_dir=epochdir
    )

#### Feature matching with SuperGlue

In [29]:
# Define matching parameters
matching_quality = matching.Quality.HIGH
tile_selection = matching.TileSelection.PRESELECTION
tiling_grid = [4, 3]
tiling_overlap = 200
geometric_verification = matching.GeometricVerification.PYDEGENSAC
geometric_verification_threshold = 1
geometric_verification_confidence = 0.9999
match_dir = epoch.epoch_dir / "matching"

# Create a new matcher object
matcher = matching.SuperGlueMatcher(cfg.matching)
matcher.match(
    epoch.images[cams[0]].value,
    epoch.images[cams[1]].value,
    quality=matching_quality,
    tile_selection=tile_selection,
    grid=tiling_grid,
    overlap=tiling_overlap,
    do_viz_matches=True,
    do_viz_tiles=False,
    save_dir=match_dir,
    geometric_verification=geometric_verification,
    threshold=geometric_verification_threshold,
    confidence=geometric_verification_confidence,
)
timer.update("matching")

2023-08-29 18:20:07 | [INFO    ] Running inference on device cuda
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
2023-08-29 18:20:07 | [INFO    ] Matching by tiles...
2023-08-29 18:20:07 | [INFO    ] Matching tiles by preselection tile selection
2023-08-29 18:20:08 | [INFO    ] Matching completed.
2023-08-29 18:20:08 | [INFO    ]  - Matching tile pair (3, 2)
2023-08-29 18:20:10 | [INFO    ]  - Matching tile pair (4, 7)
2023-08-29 18:20:12 | [INFO    ]  - Matching tile pair (5, 7)
2023-08-29 18:20:15 | [INFO    ]  - Matching tile pair (5, 8)
2023-08-29 18:20:17 | [INFO    ]  - Matching tile pair (6, 6)
2023-08-29 18:20:19 | [INFO    ]  - Matching tile pair (6, 9)
2023-08-29 18:20:21 | [INFO    ]  - Matching tile pair (7, 6)
2023-08-29 18:20:24 | [INFO    ]  - Matching tile pair (7, 7)
2023-08-29 18:20:26 | [INFO    ]  - Matching tile pair (7, 9)
2023-08-29 18:20:28 | [INFO    ]  - Matching tile pair (7, 10)
2023-08-29 18:20:31 | [INFO    ]  - Matching tile pair (8, 7

Extract the matched features from the Matcher object and save them in the current Epoch object

In [6]:
# Define a dictionary with empty Features objects for each camera, which will be filled with the matched keypoints, descriptors and scores
f = {cam: icepy4d_classes.Features() for cam in cams}

# Stack matched keypoints, descriptors and scores into Features objects
f[cams[0]].append_features_from_numpy(
    x=matcher.mkpts0[:, 0],
    y=matcher.mkpts0[:, 1],
    descr=matcher.descriptors0,
    scores=matcher.scores0,
)
f[cams[1]].append_features_from_numpy(
    x=matcher.mkpts1[:, 0],
    y=matcher.mkpts1[:, 1],
    descr=matcher.descriptors1,
    scores=matcher.scores1,
)

# Store the dictionary with the features in the Epoch object
epoch.features = f

#### Scene reconstruction

First, perform Relative orientation of the two cameras by using the matched features and the a-priori camera interior orientation.

In [15]:
# Initialize RelativeOrientation class with a list containing the two
# cameras and a list contaning the matched features location on each camera.
relative_ori = sfm.RelativeOrientation(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
)
relative_ori.estimate_pose(
    threshold=cfg.matching.pydegensac_threshold,
    confidence=0.999999,
    scale_factor=np.linalg.norm(
        cfg.georef.camera_centers_world[0] - cfg.georef.camera_centers_world[1]
    ),
)
# Store result in camera 1 object
epoch.cameras[cams[1]] = relative_ori.cameras[1]

# Update timer
timer.update("relative orientation")

2023-08-29 17:34:47 | [INFO    ] Relative Orientation - valid points: 1775/2018
2023-08-29 17:34:47 | [INFO    ] Relative orientation Succeded.


Triangulate points into the object space

In [14]:
triang = sfm.Triangulate(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
)
points3d = triang.triangulate_two_views(
    compute_colors=True, image=images[cams[1]].read_image(ep).value, cam_id=1
)

# Update timer
timer.update("triangulation")

2023-08-29 17:34:37 | [INFO    ] Point triangulation succeded: 1.0.
2023-08-29 17:34:37 | [INFO    ] Point colors interpolated


Perform an absolute orientation of the current solution (i.e., cameras' exterior orientation and 3D points) by using the ground control points.

The coordinates of the two cameras are used as additional ground control points for estimating a Helmert transformation.

In [13]:
# Get targets available in all cameras. The Labels of valid targets are returned as second element by the get_image_coor_by_label() method
valid_targets = epoch.targets.get_image_coor_by_label(
    cfg.georef.targets_to_use, cam_id=0
)[1]

# Check if the same targets are available in all cameras
for id in range(1, len(cams)):
    assert (
        valid_targets
        == epoch.targets.get_image_coor_by_label(
            cfg.georef.targets_to_use, cam_id=id
        )[1]
    ), f"""epoch {ep} - {epoch_dict[ep]}: 
    Different targets found in image {id} - {images[cams[id]][ep]}"""

# Check if there are enough targets
assert len(valid_targets) > 1, f"Not enough targets found in epoch {ep}"

# If not all the targets defined in the config file are found, log a warning and use only the valid targets
if valid_targets != cfg.georef.targets_to_use:
    logger.warning(f"Not all targets found. Using onlys {valid_targets}")

# Get image and object coordinates of valid targets
image_coords = [
    epoch.targets.get_image_coor_by_label(valid_targets, cam_id=id)[0]
    for id, cam in enumerate(cams)
]
obj_coords = epoch.targets.get_object_coor_by_label(valid_targets)[0]

# Perform absolute orientation
abs_ori = sfm.Absolute_orientation(
    (epoch.cameras[cams[0]], epoch.cameras[cams[1]]),
    points3d_final=obj_coords,
    image_points=image_coords,
    camera_centers_world=cfg.georef.camera_centers_world,
)
T = abs_ori.estimate_transformation_linear(estimate_scale=True)
points3d = abs_ori.apply_transformation(points3d=points3d)
for i, cam in enumerate(cams):
    epoch.cameras[cam] = abs_ori.cameras[i]

# Convert the 3D points to an icepy4d Points object
pts = icepy4d_classes.Points()
pts.append_points_from_numpy(
    points3d,
    track_ids=epoch.features[cams[0]].get_track_ids(),
    colors=triang.colors,
)

# Store the points in the Epoch object
epoch.points = pts

# Update timer
timer.update("absolute orientation")

2023-08-29 17:34:18 | [WARNING ] Warning: target T2 is not present on camera 0.
2023-08-29 17:34:18 | [WARNING ] Warning: target F10_2 is not present on camera 0.
2023-08-29 17:34:18 | [WARNING ] Warning: target T2 is not present on camera 1.
2023-08-29 17:34:18 | [WARNING ] Warning: target F10_2 is not present on camera 1.
2023-08-29 17:34:18 | [WARNING ] Not all targets found. Using onlys ['F2', 'F12', 'F13']
2023-08-29 17:34:18 | [INFO    ] Point triangulation succeded: 1.0.


Save the current Epoch object as a pickle file.

In [9]:
# Save epoch as a pickle object
if epoch.save_pickle(f"{epoch.epoch_dir}/{epoch}.pickle"):
    logger.info(f"{epoch} saved successfully")
else:
    logger.error(f"Unable to save {epoch}")

2023-08-29 18:32:30 | [INFO    ] 2022-05-01_14:01:15 saved successfully


#### Big loop over the epoches

Stack all the processing of a single epoch into a function and iterate over all the epoches


In [12]:
# Define processing for single epoch
def process_epoch(epoch, cfg, timer) -> Epoch:

    cams = cfg.cams
    epochdir = epoch.epoch_dir
    match_dir = epochdir / "matching"
   
    # Matching
    matching_quality = matching.Quality.HIGH
    tile_selection = matching.TileSelection.PRESELECTION
    tiling_grid = [4, 3]
    tiling_overlap = 200
    geometric_verification = matching.GeometricVerification.PYDEGENSAC
    geometric_verification_threshold = 1
    geometric_verification_confidence = 0.9999
    matcher = matching.SuperGlueMatcher(cfg.matching)    
    matcher.match(
        epoch.images[cams[0]].value,
        epoch.images[cams[1]].value,
        quality=matching_quality,
        tile_selection=tile_selection,
        grid=tiling_grid,
        overlap=tiling_overlap,
        do_viz_matches=True,
        do_viz_tiles=False,
        save_dir=match_dir,
        geometric_verification=geometric_verification,
        threshold=geometric_verification_threshold,
        confidence=geometric_verification_confidence,
    )
    f = {cam: icepy4d_classes.Features() for cam in cams}
    f[cams[0]].append_features_from_numpy(
        x=matcher.mkpts0[:, 0],
        y=matcher.mkpts0[:, 1],
        descr=matcher.descriptors0,
        scores=matcher.scores0,
    )
    f[cams[1]].append_features_from_numpy(
        x=matcher.mkpts1[:, 0],
        y=matcher.mkpts1[:, 1],
        descr=matcher.descriptors1,
        scores=matcher.scores1,
    )
    epoch.features = f
    timer.update("matching")
    
    # Relative orientation
    relative_ori = sfm.RelativeOrientation(
    [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
    [
        epoch.features[cams[0]].kpts_to_numpy(),
        epoch.features[cams[1]].kpts_to_numpy(),
    ],
    )
    relative_ori.estimate_pose(
    threshold=cfg.matching.pydegensac_threshold,
    confidence=0.999999,
    scale_factor=np.linalg.norm(
        cfg.georef.camera_centers_world[0] - cfg.georef.camera_centers_world[1]
    ),
    )
    epoch.cameras[cams[1]] = relative_ori.cameras[1]
    timer.update("relative orientation")

    # Triangulation
    triang = sfm.Triangulate(
        [epoch.cameras[cams[0]], epoch.cameras[cams[1]]],
        [
            epoch.features[cams[0]].kpts_to_numpy(),
            epoch.features[cams[1]].kpts_to_numpy(),
        ],
    )
    points3d = triang.triangulate_two_views(
        compute_colors=True, image=images[cams[1]].read_image(ep).value, cam_id=1
    )
    timer.update("triangulation")    

    # Absolute orientation
    valid_targets = epoch.targets.get_image_coor_by_label(
        cfg.georef.targets_to_use, cam_id=0
    )[1]
    for id in range(1, len(cams)):
        assert (
            valid_targets
            == epoch.targets.get_image_coor_by_label(
                cfg.georef.targets_to_use, cam_id=id
            )[1]
        ), f"""epoch {ep} - {epoch_dict[ep]}: 
        Different targets found in image {id} - {images[cams[id]][ep]}"""
    assert len(valid_targets) > 1, f"Not enough targets found in epoch {ep}"
    if valid_targets != cfg.georef.targets_to_use:
        logger.warning(f"Not all targets found. Using onlys {valid_targets}")

    image_coords = [
        epoch.targets.get_image_coor_by_label(valid_targets, cam_id=id)[0]
        for id, cam in enumerate(cams)
    ]
    obj_coords = epoch.targets.get_object_coor_by_label(valid_targets)[0]

    abs_ori = sfm.Absolute_orientation(
        (epoch.cameras[cams[0]], epoch.cameras[cams[1]]),
        points3d_final=obj_coords,
        image_points=image_coords,
        camera_centers_world=cfg.georef.camera_centers_world,
    )
    T = abs_ori.estimate_transformation_linear(estimate_scale=True)
    points3d = abs_ori.apply_transformation(points3d=points3d)
    for i, cam in enumerate(cams):
        epoch.cameras[cam] = abs_ori.cameras[i]

    pts = icepy4d_classes.Points()
    pts.append_points_from_numpy(
        points3d,
        track_ids=epoch.features[cams[0]].get_track_ids(),
        colors=triang.colors,
    )
    epoch.points = pts
    timer.update("absolute orientation")    

In [11]:
epoch = inizialization.initialize_epoch(
    cfg=cfg, images=images, epoch_id=ep, epoch_dir=epochdir
    )
process_epoch(epoch, cfg, timer)

2023-08-29 18:36:01 | [INFO    ] Running inference on device cuda
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
2023-08-29 18:36:03 | [INFO    ] Matching by tiles...
2023-08-29 18:36:03 | [INFO    ] Matching tiles by preselection tile selection
2023-08-29 18:36:03 | [INFO    ] Matching completed.
2023-08-29 18:36:03 | [INFO    ]  - Matching tile pair (3, 2)
2023-08-29 18:36:06 | [INFO    ]  - Matching tile pair (4, 7)
2023-08-29 18:36:08 | [INFO    ]  - Matching tile pair (5, 7)
2023-08-29 18:36:10 | [INFO    ]  - Matching tile pair (5, 8)
2023-08-29 18:36:13 | [INFO    ]  - Matching tile pair (6, 6)
2023-08-29 18:36:15 | [INFO    ]  - Matching tile pair (6, 9)
2023-08-29 18:36:17 | [INFO    ]  - Matching tile pair (7, 6)
2023-08-29 18:36:20 | [INFO    ]  - Matching tile pair (7, 7)
2023-08-29 18:36:22 | [INFO    ]  - Matching tile pair (7, 9)
2023-08-29 18:36:24 | [INFO    ]  - Matching tile pair (7, 10)
2023-08-29 18:36:27 | [INFO    ]  - Matching tile pair (8, 7

In [ ]:
# Add epoch to epoches object
epoches.add_epoch(epoch)

In [ ]:
logger.info("------------------------------------------------------")
logger.info("Processing started:")
timer = icepy4d_utils.AverageTimer()
iter = 0  # necessary only for printing the number of processed iteration
